In [2]:
from dash import Dash, dcc, html

from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

import plotly.express as px
import pandas as pd

ski_resorts = pd.read_csv("European_Ski_Resorts.csv").drop("Unnamed: 0", axis=1)

lift_type_by_country = (
    ski_resorts
    .groupby("Country", as_index=False)
    .agg({"SurfaceLifts":"sum", "ChairLifts":"sum", "GondolaLifts":"sum", "TotalLifts":"sum"})
    .sort_values("TotalLifts", ascending=False)
)

app = Dash(__name__)

app.layout = html.Div([
    html.H2(id="title"),
    dcc.Checklist(
        id="Feature Checklist",
        options=["Has Snow Park", "Has Night Ski"],
        value=["Has Snow Park", "Has Night Ski"]
),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('graph', 'figure'),
    Input("Feature Checklist", "value"),
)

def plot_lift_number(features):
    if features == []:
        df = (ski_resorts
              .groupby("Country", as_index=False)
              .agg({"TotalLifts":"sum"})
             )
    elif len(features) == 2:
        df = (ski_resorts
              .query("Snowparks == 'Yes' and NightSki == 'Yes'")
              .groupby("Country", as_index=False)
              .agg({"TotalLifts":"sum"})
             )
    elif features == ["Has Snow Park"]:
        df = (ski_resorts
              .query("Snowparks == 'Yes'")
              .groupby("Country", as_index=False)
              .agg({"TotalLifts":"sum"})
             )
    else:
        df = (ski_resorts
              .query("NightSki == 'Yes'")
              .groupby("Country", as_index=False)
              .agg({"TotalLifts":"sum"})
             )
    fig = px.choropleth(
        df,
        locations="Country",
        color="TotalLifts",
        locationmode="country names",
        scope="europe",
    ).update_geos(fitbounds="locations").update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    

    return fig
    
if __name__ == "__main__":
    app.run_server(
        debug=True,      
)